In [1]:
import openai
openai.api_key=""

In [2]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role":"system","content":"You are a helpful AI Assistant."}])

InvalidRequestError: The model: `gpt-4` does not exist

In [73]:
response["choices"][0]["message"]["content"]

'How may I assist you today?'

In [4]:
response = openai.Moderation.create(
            input="I wanna kill myself",
            )

In [3]:
models = openai.Model.list()


In [4]:
[m["id"] for m in models["data"] if "gpt" in m["id"]]

['gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0613']

In [3]:
from doc_utils import VectorDB
from sqlalchemy.orm import Session

In [4]:
vector_db = VectorDB()

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [46]:
vector_db.similarity_search_with_score("What is the title of this protocol?", filter={"source":"protocols/NCT00467987.pdf"})

[(Document(page_content='and risk of developing type 2 diabetes mel litus. Jama  286:327 -334, 2001', metadata={'source': 'protocols/NCT00467987.pdf', 'page': 17}),
  0.5237897211297184),
 (Document(page_content='Page 4 of 56  IRB Version: JAN2016   Location: 1000 Youngs Road, Suite 105  \nWilliamsville NY 14221  \nDepartment: Diabetes Endocrinology & Metabolism         \n    \n1.0 Objectives  \n1.1 Describe the purpose, specific aims, or objectives of this research.  \nResponse: We have shown that the prevalence of hypogonadotrophic hypogonadism (HH) \nin type 2 diabetic (DM) patient population may be around 35%. We have also shown that \nin an age and glycemia matched group of type 1 diabetic patients, there is no significa nt \nhypogonadotrophic hypogonadism. HH in type 2 diabetics is associated with a normal \nrelease of LH and FSH following GnRH administration. Thus, the defect in testosterone \nsecretion is specific to type 2 diabetes and is likely to be at the hypothalamic lev e

In [35]:
" ".join([c[0].metadata["source"]+" "+str(c[0].metadata["page"])+ "\n\n" +c[0].page_content for c in vector_db.similarity_search_with_score("Abhiraj")])

'protocols/NCT00217022.pdf 2\n\n2 I. A bstr act: protocols/NCT00217022.pdf 4\n\nGastr o 1 9 9 8; 9 3: 1 3 0 0). protocols/NCT00217022.pdf 3\n\n3 E C) is desi g ne d t o deli ver dr u g ma xi mall y t o t he distal ile u m a n d pr o xi mal c ol o n, c o m pare d protocols/NCT00217022.pdf 1\n\nA R a n d o mize d, D o u ble -Bli n d, Pl ace b o -C o ntr olle d Tri al of \nB u des o ni de F or T he Tre at me nt Of Acti ve L y m p h oc ytic \nC olitis  \n \n          \nPri nci ple I n v esti gat or:  D arrell S. Par di, M D  \n \nC oi n vesti gat ors: Willia m J. Sa n d b or n. M D \nLisa A. B oar d ma n, M D \nPeter W. Carr yer, M D \nJ o nat ha n E. Clai n, M D \nLa wre n ce J. E g a n, M D  \nE d war d V. L oft us, M D \nKe n net h W. Sc hr oe der, M D \nT h o mas C. S m yr k, M D \nWillia m J. Tre mai ne, M D  Ala n R. Zi ns miester, P h D \nPatricia P. Ka m mer  \n \n \n  \nDi visi o n of Gastr oe nter ol o g y a n d He pat ol o g y, Ma y o Cli nic a n d \nF o u n dati o n, R oc heste

In [14]:
vector_db.create_tables_if_not_exists()

In [15]:
vector_db.connect()

In [8]:
collection.get_by_name(name="protocols")

TypeError: CollectionStore.get_by_name() missing 1 required positional argument: 'session'

In [9]:
vector_db.create_vector_extension()

AttributeError: 'VectorDB' object has no attribute 'logger'

In [1]:
from doc_utils import ProcessDocument

In [45]:
len(proto)

36

In [60]:
import re

def identify_sections(text, page):
    section_pattern = r'^(\d+(\.\d+)*)\s*(.*?)$'
    subsection_pattern = r'^(\d+(\.\d+)*)\s*(.*?)$'

    sections = []
    subsections = []
    current_section = None

    lines = text.split('\n')
    for line in lines:
        line = line.strip()

        section_match = re.match(section_pattern, line)
        subsection_match = re.match(subsection_pattern, line)

        if section_match:
            section_number = section_match.group(1)
            section_name = section_match.group(3)
            current_section = {"number": section_number, "name": section_name,
                    "page": page, "subsections": []}
            sections.append(current_section)
        elif subsection_match:
            subsection_number = subsection_match.group(1)
            subsection_name = subsection_match.group(3)
            if current_section is not None:
                current_section["subsections"].append({
                    "number": subsection_number, 
                    "name": subsection_name})

    return sections


In [61]:
def protocol(protocol_number):
    proto = ProcessDocument(f"/Users/ansh/langchain-setup/downloaded_protocols/{protocol_number}").load_and_chunk(no_chunk=True)
    sections=[]
    for i,page in enumerate(proto):
        sections.extend(identify_sections(proto[i].page_content, i))
    return sections

In [63]:
protocol("NCT04223635.pdf")

[{'number': '2',
  'name': '1 1   M  T.   AI  R  Y   R  O  A  D',
  'page': 0,
  'subsections': []},
 {'number': '3',
  'name': '0  mi n ut es [  mi n]  pri or t o  d osi n g)  a n d  at  0. 5,  1,  1. 5,  2,  2. 5, 3,  3. 5,  4,  4. 5,  5,  6,  8,  1 0,  1 2,  2 4,  3 6,  4 8,  7 2,  9 6,  1 2 0,  1 4 4, a n d  1 6 8  h  p ost  d os e.',
  'page': 6,
  'subsections': []},
 {'number': '2', 'name': 't o  4 0', 'page': 7, 'subsections': []},
 {'number': '1',
  'name': '. I  N T  R  O  D  U  C TI  O  N ...........................................................................................................2 0',
  'page': 10,
  'subsections': []},
 {'number': '1',
  'name': '. 1. B a c k gr o u n d   ......................................................................................................................2 0',
  'page': 10,
  'subsections': []},
 {'number': '1',
  'name': '. 2. D at a  S u  m  m ar y   ..........................................................................

In [77]:
import json

In [82]:
with open("prompts.json") as p:
    prompt = json.load(p)

In [87]:
prompt["messages"][1]["content"]="a"

In [88]:
prompt

{'messages': [{'role': 'system',
   'content': "You are a clinical trial protocol analyst. You analyze protocol documents and provide information from the protocol to the clinical trial team. When you do not have the information being asked for, you say you don't know"},
  {'role': 'user', 'content': 'a'}]}

In [2]:
from unstructured.partition.auto import partition

elements = partition(filename="/Users/ansh/llm_app_/protocols/NCT00363649.pdf")
print("\n\n".join([str(el) for el in elements]))

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with another strategy.
Falling back to partitioning with ocr_only.


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [1]:
import sys
import os
sys.path.append('..')

from db.utils import VectorDB, ProcessDocument
vector_db = VectorDB()



In [2]:
vector_db.get_document_names()

['protocol.pdf',
 'NCT00363649.pdf',
 'NCT05216172.pdf',
 'NCT00534118.pdf',
 'NCT02453191.pdf',
 'NCT00799266.pdf',
 'NCT03462927.pdf',
 'NCT03218592.pdf',
 'NCT00998010.pdf']